# Linear Solves in UnifyML

[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/holl-/UnifyML/blob/main/docs/Linear_Solves.ipynb)
&nbsp; • &nbsp; [🌐 **UnifyML**](https://github.com/holl-/UnifyML)
&nbsp; • &nbsp; [📖 **Documentation**](https://holl-.github.io/UnifyML/)
&nbsp; • &nbsp; [🔗 **API**](https://holl-.github.io/UnifyML/unifyml)
&nbsp; • &nbsp; [**▶ Videos**]()
&nbsp; • &nbsp; [<img src="images/colab_logo_small.png" height=4>](https://colab.research.google.com/github/holl-/UnifyML/blob/main/docs/Examples.ipynb) [**Examples**](https://holl-.github.io/UnifyML/Examples.html)

Linear solves are a vital part in many simulations and machine learning applications.
UnifyML provides an easy-to-use interface for performing linear solves that supports backpropagation via implicit gradients.
Dense, sparse, and matrix-free linear systems can be solved this way.


In [1]:
%%capture
!pip install unifyml

In [10]:
from unifyml import math
from unifyml.math import wrap, channel, dual, Solve, const_vec

Linear solves and sparse matrices are supported on all backends.
Feel free to choose the below line to use `jax`, `tensorflow` or `numpy` instead.

In [17]:
math.use('torch')

torch

We can perform a linear solve by passing a matrix `A`, right-hand-side vector `b` and initial guess `x0` to [`solve_linear()`](unifyml/math/#unifyml.math.solve_linear).
The [dual dimensions of the matrix](Matrices.html#Primal-and-Dual-Dimensions) must match the initial guess and the primal dimensions must match the right-hand-side.

In [18]:
A = wrap([[0, 1], [1, 0]], channel('b_vec'), dual('x_vec'))
b = wrap([2, 3], channel('b_vec'))
x0 = wrap([0, 0], channel('x_vec'))
math.solve_linear(A, b, Solve(x0=x0))

(3.000, 2.000) along b_vecᶜ

UnifyML implements multiple algorithms to solve linear systems, such as the conjugate gradient method (`CG`) and the stabilized bi-conjugate gradient method (`biCG`).
All SciPy solvers are also available.
For a full list, see [here](unifyml/math/#unifyml.math.solve_linear).

In [24]:
math.solve_linear(A, b, Solve('CG', x0=x0))

(3.000, 2.000) along b_vecᶜ

In [25]:
math.solve_linear(A, b, Solve('biCG-stab', x0=x0))

(3.000, 2.000) along b_vecᶜ

In [28]:
math.solve_linear(A, b, Solve('scipy-GMres', x0=x0))

(3.000, 2.000) along b_vecᶜ

## Matrix-free Solves

Instead of passing a matrix, you can also specify a linear Python function that computes the matrix-vector product.
This will typically be slower unless the function is compiled to a matrix.

In [21]:
def linear_function(x):
    return x * (2, 1)

math.solve_linear(linear_function, b, Solve(x0=x0))

(1.000, 3.000) along b_vecᶜ

## Explicit Matrices from Python Functions

UnifyML can also [build an explicit matrix representation](Matrices.html#Building-Matrices-from-Linear-Functions) of the provided Python function.
You can do this either by explicitly obtaining the matrix first using [`matrix_from_function`](unifyml/math#unifyml.math.matrix_from_function) or by annotating the linear function with
[`jit_compile_linear`](unifyml/math#unifyml.math.jit_compile_linear).
If the function adds a constant offset to the output, this will automatically be subtracted from the right-hand-side vector.

In [23]:
from unifyml.math import jit_compile_linear

math.solve_linear(jit_compile_linear(linear_function), b, Solve(x0=x0))

(1.000, 2.000, 1.500, 3.000) (b_vecᶜ=2, x_vecᶜ=2)

## Preconditioned Linear Solves

Preconditioner at jit time if matrix NumPy
[](Matrices.html#Building-Matrices-from-Linear-Functions))
[](NumPy_Constants.html)

## Implicit Differentiation

## Matrix Gradients

UnifyML can also compute gradients for the (sparse) matrix used in a linear solve, which allows differentiating w.r.t. parameters that influenced the matrix values via backpropagation.


## Obtaining Additional Information about a Solve

`SolveTape`

## Further Reading

We will upload a whitepaper to the ArXiv shortly, describing the implemented algorithms.

&nbsp; • &nbsp; [🌐 **UnifyML**](https://github.com/holl-/UnifyML)
&nbsp; • &nbsp; [📖 **Documentation**](https://holl-.github.io/UnifyML/)
&nbsp; • &nbsp; [🔗 **API**](https://holl-.github.io/UnifyML/unifyml)
&nbsp; • &nbsp; [**▶ Videos**]()
&nbsp; • &nbsp; [<img src="images/colab_logo_small.png" height=4>](https://colab.research.google.com/github/holl-/UnifyML/blob/main/docs/Examples.ipynb) [**Examples**](https://holl-.github.io/UnifyML/Examples.html)